In [0]:
# Define schema
from pyspark.sql.types import  StructType, StructField, IntegerType, StringType

schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                            StructField("driverId", IntegerType(), True),
                            StructField("lap", IntegerType(), True),
                            StructField("position", IntegerType(), True),
                            StructField("time", StringType(), True),
                            StructField("milliseconds", IntegerType(), True)])

In [0]:
# Read csv files

laptimes_df = spark.read.csv("/mnt/formula1dataplatformdl/raw/lap_times", schema=schema)
display(laptimes_df)

In [0]:
# Process data
from pyspark.sql.functions import current_timestamp
laptimes_processed = laptimes_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumn("ingestion_date", current_timestamp())

display(laptimes_processed)                    

In [0]:
# Write df into parquet
laptimes_processed.write.parquet("/mnt/formula1dataplatformdl/processed/lap_times", mode="overwrite")

In [0]:
# Read data to check if it have been properly written
display(spark.read.parquet("/mnt/formula1dataplatformdl/processed/lap_times"))